In [ ]:
import geopandas as gpd
import pandas as pd
import geopy as gpy
import matplotlib.pyplot as plt
import contextily as ctx
%matplotlib inline

In [ ]:
pd.set_option('display.max_columns', None)

# Point frequency map showing the properties with 20 or more code violations

## Read data into geoDataFrame

In [ ]:
url = 'https://data.buffalony.gov/resource/ivrf-k9vm.geojson?$limit=1000000'
code_v = codev_extra = gpd.read_file(url)
code_v.shape

## Remove record with no geometry

In [ ]:
nrows = code_v.shape[0]
code_v = code_v.loc[code_v.geometry.notnull()]
krows = code_v.shape[0]
removed = nrows - krows
pctremoved = (removed/nrows)*100
print ("Original number of rows = {}\nNumber of rows missing coordinates = {},\nPercent missing data = {:6.1f}%".format(nrows, removed, pctremoved))
if pctremoved > 10:
    print("\nWARNING: Percent missing location data exceeds recommended limit!")

## New field that combines the longitude and latitude value into a single string

In [ ]:
long_lat = (code_v['longitude'] + code_v['latitude']).values
code_v = code_v.assign(newloc = long_lat)

## Count number of times each point value occurs

In [ ]:
numinc = code_v.newloc.value_counts().rename_axis('long_lat').to_frame('counts')
numinc

## Merging DataFrame code_v and numinc

In [ ]:
codepts = pd.merge(code_v, numinc, left_on = 'newloc', right_on = 'long_lat')
codepts = codepts.drop_duplicates(subset = 'newloc')
codepts.shape

## Histogram plot

In [ ]:
plt.hist(codepts.counts, bins = [5, 10, 25, 50, 100, 500, 1000])

## Finding locations with over 20 code violation calls

In [ ]:
codev_pts = codepts.loc[codepts.counts >= 20].copy()
codev_pts.shape

## Finding the total number of code violations where each location has over 20 code violation calls

In [ ]:
tot_code = codev_pts.counts.sum()
tot_code

## Reading in community data

In [ ]:
url_neigh = 'https://data.buffalony.gov/resource/g7bi-nz8b.geojson?$limit=1000000'
hoods = gpd.read_file(url_neigh)
hoods.shape

## Converting to epsg: 3857

In [ ]:
codev_pts = codev_pts.to_crs('epsg:3857')
hoods = hoods.to_crs('epsg:3857')
code_v = code_v.to_crs('epsg:3857')
codepts = codepts.to_crs('epsg:3857')

## Setting circle size to be a proportion of the code violation calls count to the total code violation calls

In [ ]:
cir_size = (codev_pts.counts/tot_code)*300000 #Chose arbitary factor to make everything more legible
cir_size

## Create a point frequency map showing the properties with 20 or more code violations

In [ ]:
hoods_poly = hoods.plot(figsize = (10, 10), alpha = 0.5, edgecolor = 'black', color = 'honeydew')
ctx.add_basemap(hoods_poly,source = ctx.providers.Stamen.TonerLite)

codev_pts.plot(ax = hoods_poly, edgecolor = 'black', alpha = 0.3, color = 'dodgerblue', marker = 'o', markersize = cir_size)

# Bokeh interactive map of properties with 20 or more code violations

In [ ]:
from bokeh.tile_providers import CARTODBPOSITRON, get_provider
tileProvider = get_provider('CARTODBPOSITRON_RETINA')
from bokeh.io import output_notebook, show, output_file, save
from bokeh.plotting import figure
from bokeh.models import (HoverTool, GeoJSONDataSource, LogColorMapper, ColorBar)
from bokeh.transform import linear_cmap,log_cmap
import bokeh.palettes

In [ ]:
output_notebook()

In [ ]:
TOOLS = "pan, wheel_zoom, box_zoom, reset, save"

In [ ]:
codev_pts['ccsize'] = (codev_pts.counts/codev_pts.counts.sum()) * 10000

In [ ]:
f = figure(title = 'Properties with 20 or more code violations', tools = TOOLS,\
plot_width = 700, plot_height = 700,\
outline_line_color = None,\
min_border = 0, min_border_left = 0, min_border_right = 0,\
min_border_top = 0, min_border_bottom = 0)

In [ ]:
f.add_tile(tileProvider)
f.title.text_font_style = 'italic'
f.title.text_font_size = '14pt'
f.axis.visible = False

In [ ]:
from bokeh.palettes import Turbo256

In [ ]:
point_source = GeoJSONDataSource(geojson = codev_pts.to_json())
poly_source = GeoJSONDataSource(geojson = hoods.to_json())

areas = f.patches('xs', 'ys', source = poly_source, fill_color = 'honeydew', fill_alpha = 0.5, line_color = 'black', line_width = 0.5)

circles = f.circle('x', 'y', source = point_source, size = 'ccsize', fill_alpha = 0.3, fill_color = 'dodgerblue', line_color = 'black')

In [ ]:
c_hover = HoverTool(renderers = [circles])
c_hover.point_policy = 'follow_mouse'
c_hover.tooltips = [('Neighborhood Name', '@neighborhood'), ('Code Violations', '@counts')]
f.add_tools(c_hover)

output_file('Properties with 20 or more code violations.html', title = 'Properties with 20 or more code violations')

show(f)

# Point distribution map showing the number of code violations in each neighborhood

## Performing spatial join of data frame code_v and hoods

In [ ]:
codev_neigh = gpd.sjoin(code_v, hoods, how = 'inner', op = 'intersects')

In [ ]:
codev_neigh.nbhdname.count()

## Get a count of the number of points in each polygon

In [ ]:
ct = codev_neigh.nbhdname.groupby(codev_neigh['nbhdname']).count().sort_values(ascending = False)

ctdf = ct.to_frame(name = 'counts').reset_index()

## Merge counts by neighborhood with the polygon geoDataFrame and determine proportional circle size

In [ ]:
neigh_code = pd.merge(hoods, ctdf)
minct = neigh_code['counts'].min()
mincir = 75
neigh_code['circle size'] = (neigh_code['counts']/minct * mincir)
neigh_code['circle size'] = neigh_code['circle size'].round(0).astype(int)

## Get centroid of each polygon and make the centroid the active geometry field for the geoDataFrame

In [ ]:
neigh_code['centroids'] = neigh_code['geometry'].centroid
neigh_code = neigh_code.set_geometry('centroids')

## Set neigh_code to epsg:3857

In [ ]:
neigh_code = neigh_code.to_crs('epsg:3857')

## Create a point distribution map showing the number of code violations in each neighborhood

In [ ]:
hoods_poly = hoods.plot(figsize = (10, 10), alpha = 0.5, color = 'azure', edgecolor = 'black')
ctx.add_basemap(hoods_poly, source = ctx.providers.Stamen.TonerLite)

neigh_code.plot(ax = hoods_poly, markersize = neigh_code['circle size'], alpha = 0.3, color = 'lawngreen', marker = 'o', edgecolor = 'black');

# Bokeh interactive map of number of code violations in each neighborhood

In [ ]:
neigh_code['ccsize'] = neigh_code['circle size']/20

In [ ]:
f = figure(title = 'Number of code violations in each neighborhood', tools = TOOLS,\
plot_width = 750, plot_height = 750,\
outline_line_color = None,\
min_border = 0, min_border_left = 0, min_border_right = 0,\
min_border_top = 0, min_border_bottom = 0)

In [ ]:
f.add_tile(tileProvider)
f.title.text_font_style = 'italic'
f.title.text_font_size = '14pt'
f.axis.visible = False
from bokeh.palettes import Turbo256

neigh_code = neigh_code.drop('geometry', axis = 1).copy()

point_source = GeoJSONDataSource(geojson = neigh_code.to_json())
poly_source = GeoJSONDataSource(geojson = hoods.to_json())

areas = f.patches('xs', 'ys', source = poly_source, fill_color = 'azure', fill_alpha = 0.5, line_color = 'black', line_width = 0.5)

circles = f.circle('x', 'y', source = point_source, size = 'ccsize', fill_alpha = 0.3, fill_color = 'lawngreen', line_color = 'black')

c_hover = HoverTool(renderers = [circles])
c_hover.point_policy = 'follow_mouse'
c_hover.tooltips = [('Neighborhood Name', '@nbhdname'), ('Code Violations', '@counts')]

f.add_tools(c_hover)

output_file('Neighborhood Code Violations.html', title = 'Neighborhood Code Violations')

show(f)

# 3. Create a choropleth map showing code violations per sq. mile for each neighborhood

### Merge counts by neighborhood with the polygon geoDataFrame

In [ ]:
neigh_code = pd.merge(hoods, ctdf)

### Determining value for density based on neighborhood count over the sq. mileage of neighborhood

In [ ]:
neigh_code['codev_density'] = neigh_code['counts'].astype(int)/neigh_code['sqmiles'].astype(float)

### Set neigh_code and hoods to epsg:3857

In [ ]:
neigh_code = neigh_code.to_crs('epsg:3857')

### Create a choropleth map showing violations per sq. mile for each neighborhood

In [ ]:
hoods_poly = hoods.plot(figsize = (10, 10))
ctx.add_basemap(hoods_poly, source = ctx.providers.Stamen.TonerLite)

neigh_code.plot(ax = hoods_poly, figsize = (10, 10), column = neigh_code['codev_density'], cmap = 'winter', edgecolor = 'black', legend = True);

# Bokeh interactive maps

In [ ]:
f = figure(title = 'Code violations per square mile for each neighborhood', tools = TOOLS,\
plot_width = 750, plot_height = 750,\
outline_line_color = None,\
min_border = 0, min_border_left = 0, min_border_right = 0,\
min_border_top = 0, min_border_bottom = 0)

In [ ]:
f.add_tile(tileProvider)
f.title.text_font_style = 'italic'
f.title.text_font_size = '14pt'
f.axis.visible = False
from bokeh.palettes import Turbo256

In [ ]:
point_source = GeoJSONDataSource(geojson = neigh_code.to_json())
poly_source = GeoJSONDataSource(geojson = hoods.to_json())

from bokeh.palettes import brewer

palette = brewer['GnBu'][8]
palette = palette[::-1]

In [ ]:
vals = neigh_code['codev_density']

from bokeh.models import LinearColorMapper, ColorBar

color_mapper = LinearColorMapper(palette = palette, low = vals.min(), high = vals.max())
color_bar = ColorBar(color_mapper = color_mapper, label_standoff = 8, width = 500, height = 20, 
                         location = (0, 0), orientation = 'horizontal')

In [ ]:
areas = f.patches('xs', 'ys', source = point_source, fill_color = {'field': 'codev_density', 'transform': color_mapper}, fill_alpha = 0.5, line_color = 'black', line_width = 0.5)

#circles = f.circle('x', 'y', source = point_source, size = 'ccsize', fill_alpha = 0.3, fill_color = 'lawngreen', line_color = 'black')

f.add_layout(color_bar, 'below')

c_hover = HoverTool(renderers = [areas])
c_hover.point_policy = 'follow_mouse'
c_hover.tooltips = [('Neighborhood Name', '@nbhdname'), ('Code Violations', '@counts')]

f.add_tools(c_hover)

output_file('Code violations per square mile for each neighborhood.html', title = 'Code violations per square mile for each neighborhood')

show(f)

# Plots of comparison of 'longitude and latitude' and 'address and zip code'

In [ ]:
nrows = codev_extra.shape[0]
codev_extra = codev_extra.loc[codev_extra.geometry.notnull()]
krows = codev_extra.shape[0]
removed = nrows - krows
pctremoved = (removed/nrows)*100
print ("Original number of rows = {}\nNumber of rows missing coordinates = {},\nPercent missing data = {:6.1f}%".format(nrows, removed, pctremoved))
if pctremoved > 10:
    print("\nWARNING: Percent missing location data exceeds recommended limit!")

In [ ]:
long_lat_extra = (codev_extra['address'] + ' ' + codev_extra['zip']).values
codev_extra = codev_extra.assign(extra_cred = long_lat_extra)

In [ ]:
from IPython.display import display_html

In [ ]:
def side_by_side(*args):
    html_str = ''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'), raw = True)

In [ ]:
extra_num = codev_extra.extra_cred.value_counts().rename_axis('address + zip').to_frame('counts')

In [ ]:
side_by_side(numinc.head(), extra_num.head())

In [ ]:
codev_extra.drop_duplicates(subset = 'extra_cred', inplace = True)
codev_extra_pts = pd.merge(codev_extra, extra_num, left_on = 'extra_cred', right_on = 'address + zip')
codepts.shape, codev_extra_pts.shape

In [ ]:
codepts.counts.sum(), codev_extra_pts.counts.sum()

In [ ]:
cir_size1 = (codepts.counts/codepts.counts.sum())*400000
codepts = codepts.to_crs('epsg:3857')

hoods_poly = hoods.plot(figsize = (10, 10), alpha = 0.5, edgecolor = 'black', color = 'honeydew')
ctx.add_basemap(hoods_poly, source = ctx.providers.Stamen.TonerLite)

codepts.plot(ax = hoods_poly, edgecolor = 'black', alpha = 0.3, color = 'dodgerblue', marker = 'o', markersize = cir_size1);

In [ ]:
cir_size2 = (codev_extra_pts.counts/codev_extra_pts.counts.sum())*400000
codev_extra_pts = codev_extra_pts.to_crs('epsg:3857')

hoods_poly = hoods.plot(figsize = (10, 10), alpha = 0.5, edgecolor = 'black', color = 'honeydew')
ctx.add_basemap(hoods_poly, source = ctx.providers.Stamen.TonerLite)

codev_extra_pts.plot(ax = hoods_poly, edgecolor = 'black', alpha = 0.3, color = 'dodgerblue', marker = 'o', markersize = cir_size2);

# Bokeh interactive maps of previous comparison

## Interactive map using original codepts; latitude + longitude

In [ ]:
codepts['cir_size1'] = (codepts.counts/codepts.counts.sum()) * 100000

In [ ]:
f = figure(title = 'Point frequency analysis using latitude + longitude', tools = TOOLS,\
plot_width = 700, plot_height = 700,\
outline_line_color = None,\
min_border = 0, min_border_left = 0, min_border_right = 0,\
min_border_top = 0, min_border_bottom = 0)

In [ ]:
f.add_tile(tileProvider)
f.title.text_font_style ='italic'
f.title.text_font_size ='14pt'
f.axis.visible = False

point_source = GeoJSONDataSource(geojson = codepts.to_json())
poly_source = GeoJSONDataSource(geojson = hoods.to_json())

areas = f.patches('xs', 'ys', source = poly_source, fill_color = 'honeydew', fill_alpha = 0.5, line_color = 'black', line_width = 0.5)

circles = f.circle('x', 'y', source = point_source, size = 'cir_size1', line_width = 0.5, fill_alpha = 0.3, fill_color = 'dodgerblue', line_color = 'black')


c_hover = HoverTool(renderers = [circles])
c_hover.point_policy = 'follow_mouse'
c_hover.tooltips = [('Neighborhood Name', '@neighborhood'), ('Code Violations', '@counts')]
f.add_tools(c_hover)

output_file('Point frequency analysis using latitude + longitude.html', title = 'Point frequency analysis using latitude + longitude')

show(f)

## Interactive map with codev_extra_pts; address + zip

In [ ]:
codev_extra_pts['cir_size2'] = (codev_extra_pts.counts/codev_extra_pts.counts.sum()) * 100000

In [ ]:
f = figure(title = 'Point frequency analysis using address + zip', tools = TOOLS,\
plot_width = 700, plot_height = 700,\
outline_line_color = None,\
min_border = 0, min_border_left = 0, min_border_right = 0,\
min_border_top = 0, min_border_bottom = 0)

In [ ]:
f.add_tile(tileProvider)
f.title.text_font_style ='italic'
f.title.text_font_size ='14pt'
f.axis.visible = False

point_source = GeoJSONDataSource(geojson = codev_extra_pts.to_json())
poly_source = GeoJSONDataSource(geojson = hoods.to_json())

areas = f.patches('xs', 'ys', source = poly_source, fill_color = 'honeydew', fill_alpha = 0.5, line_color = 'black', line_width = 0.5)

circles = f.circle('x', 'y', source = point_source, size = 'cir_size2', line_width = 0.5, fill_alpha = 0.3, fill_color = 'dodgerblue', line_color = 'black')


c_hover = HoverTool(renderers = [circles])
c_hover.point_policy = 'follow_mouse'
c_hover.tooltips = [('Neighborhood Name', '@neighborhood'), ('Code Violations', '@counts')]
f.add_tools(c_hover)

output_file('Point frequency analysis using address + zip.html', title = 'Point frequency analysis using address + zip')

show(f)